# 🚀 Day 2 실습 1: Naive RAG 베이스라인 - 파스텔 문제 발견

## 🎯 학습 목표
- **단순 질문 vs 복합 질문**으로 Naive RAG의 한계 직접 체험
- **벡터 검색**만으로는 부족한 이유 이해
- **02-06번 개선 여정**의 출발점 설정

### 💡 핵심 시나리오
사용자: **"내일 저녁에 파스텔 공연 보고, 주말엔 호텔 체크인 날짜 바꾸고, 카페에서 체험 수업도 받고 싶어"**

이 복합 질문이 왜 어려울까요? 지금부터 확인해봅시다!

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers pandas numpy

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import pandas as pd
import numpy as np

print("✅ 라이브러리 준비 완료!")

## 📚 파스텔 문서 준비

### 동음이의어 "파스텔"의 세 얼굴
사용자가 "파스텔"이라고 하면... 무엇을 말하는 걸까요?

In [ ]:
# 파스텔 문서 + 노이즈 문서 (8개)
pastel_docs = [
    {
        "content": """파스텔 호텔 예약 안내
서울 강남구에 위치한 파스텔 호텔입니다.
- 예약 변경: 투숙 3일 전까지 가능 (위약금 없음)
- 주말 요금: 평일 대비 30% 할인""",
        "type": "hotel"
    },
    {
        "content": """파스텔 카페 메뉴
홍대 파스텔 카페는 디저트로 유명합니다.
- 예약: 단체 예약만 가능 (10명 이상)
- 주말: 마카롱 만들기 체험 (14:00-16:00)""",
        "type": "cafe"
    },
    {
        "content": """파스텔 소극장 공연 안내
대학로 파스텔 소극장에서 뮤지컬을 상영합니다.
- 티켓 예약: 인터파크, 현장 구매 가능
- 주말: 토요일 15:00 추가 공연""",
        "type": "theater"
    },
    {
        "content": """국립현대미술관 전시 정보
서울 국립현대미술관 덕수궁관 전시 안내입니다.
- 주말 관람 팁: 토요일 14:00-16:00 혼잡, 일요일 오전 10:00-12:00 한적함""",
        "type": "museum"
    },
    {
        "content": """서울 여행 블로그
미술관은 주말 오전이 베스트 타이밍.
예약은 미리미리! 특히 숙소는 필수.""",
        "type": "blog"
    },
    {
        "content": """서울역 교통 안내
KTX, 지하철 1/4호선 환승 가능.
주말 막차 시간: 자정까지 운행""",
        "type": "transport"
    },
    {
        "content": """주말 날씨 예보
이번 주말 서울 날씨는 맑음.
외출하기 좋은 날씨입니다.""",
        "type": "weather"
    },
    {
        "content": """예약 플랫폼 가이드
네이버, 카카오 예약 시스템 비교.
주말 예약은 평일보다 2배 빠르게 마감됩니다.""",
        "type": "guide"
    }
]

print("📚 문서 8개 준비 완료 (파스텔 5개 + 노이즈 3개)")
for i, doc in enumerate(pastel_docs, 1):
    print(f"  {i}. [{doc['type']}] {doc['content'].split(chr(10))[0]}")

## 🔍 Naive RAG 시스템 구축

가장 단순한 방법: **벡터 검색만** 사용

In [ ]:
# Document 객체로 변환
documents = [Document(page_content=doc["content"], metadata={"type": doc["type"]}) 
             for doc in pastel_docs]

# 임베딩 모델 + 벡터스토어 구축
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents, embeddings)

print("✅ Naive RAG 시스템 구축 완료!")
print(f"   임베딩 모델: BAAI/bge-m3")
print(f"   문서 수: {len(documents)}개")

## 🎯 실습 1: 나만의 파스텔 문서 추가하기

### 🎨 학습 목표
- 문서 구조 이해하기
- 새로운 데이터 추가 경험

### 💡 실습 내용
아래 코드에서 **여러분만의 "파스텔" 문서**를 추가해보세요!
- 파스텔 서점? 파스텔 미용실? 파스텔 피트니스?
- `type`과 `content`를 자유롭게 작성하세요

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 여러분만의 파스텔 문서를 1-2개 추가해보세요

my_new_docs = [
    {
        "content": """파스텔 서점 안내
강남역 근처 파스텔 서점입니다.
- 영업시간: 10:00-22:00
- 주말 이벤트: 토요일 저자 사인회 (15:00-17:00)""",
        "type": "bookstore"
    },
    # TODO: 여기에 여러분의 문서를 추가하세요!
    # {
    #     "content": """여기에 내용 작성""",
    #     "type": "여기에 타입 작성"
    # }
]

# 기존 문서에 추가
extended_pastel_docs = pastel_docs + my_new_docs

# 새로운 벡터스토어 구축
extended_documents = [Document(page_content=doc["content"], metadata={"type": doc["type"]}) 
                      for doc in extended_pastel_docs]
extended_vectorstore = FAISS.from_documents(extended_documents, embeddings)

print(f"✅ 확장된 문서 개수: {len(extended_pastel_docs)}개")
print(f"   (기존 {len(pastel_docs)}개 + 추가 {len(my_new_docs)}개)\n")

# 여러분의 문서가 잘 검색되는지 테스트
test_query = "파스텔 서점"  # TODO: 여러분의 문서와 관련된 질문으로 수정하세요
results = extended_vectorstore.similarity_search(test_query, k=1)
print(f"🔍 질문: {test_query}")
print(f"✅ 검색 결과: [{results[0].metadata.get('type')}] {results[0].page_content.split(chr(10))[0]}")

## 🧪 테스트 1: 단순 질문 (성공 케이스)

먼저 간단한 질문으로 RAG가 잘 동작하는지 확인해봅시다:

In [ ]:
# 단순 질문 2개 테스트
simple_queries = [
    "파스텔 호텔 예약 변경 방법은?",
    "주말에 미술관 언제 가면 좋아?"
]

for query in simple_queries:
    print(f"❓ 질문: {query}")
    results = vectorstore.similarity_search(query, k=1)
    
    doc = results[0]
    doc_type = doc.metadata.get("type", "unknown")
    print(f"✅ 검색 성공: [{doc_type.upper()}] {doc.page_content.split(chr(10))[0]}")
    print()

## 🎯 실습 2: 나만의 질문 만들어보기

### 🎨 학습 목표
- 다양한 질문 패턴 실험
- 단순 질문의 성공률 체험

### 💡 실습 내용
아래에서 **여러분만의 질문 3개**를 작성하고 검색 결과를 확인해보세요!

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 여러분만의 질문 3개를 작성하세요

my_queries = [
    "파스텔 카페 마카롱 만들기는 언제 해?",  # 예시 1
    "호텔 예약을 변경하려면?",               # 예시 2
    "여기에 세 번째 질문을 작성하세요"        # TODO: 수정 필요
]

print("🔍 나만의 질문 테스트 결과\n")
print("="*50)

for i, query in enumerate(my_queries, 1):
    results = vectorstore.similarity_search(query, k=1)
    doc = results[0]
    doc_type = doc.metadata.get("type", "unknown")
    first_line = doc.page_content.split("\n")[0]
    
    print(f"\n질문 {i}: {query}")
    print(f"답변: [{doc_type.upper()}] {first_line}")
    print(f"내용: {doc.page_content[:60]}...")

print("\n" + "="*50)
print("\n💡 관찰해보세요:")
print("  - 어떤 질문이 가장 정확한 답을 찾았나요?")
print("  - 실패한 질문이 있다면 왜 실패했을까요?")

## 🧪 테스트 2: 복합 질문 (실패 케이스)

이제 **3가지 정보를 동시에** 요구하는 질문을 던져봅시다:

In [ ]:
# 복합 질문 (3가지 정보 필요)
complex_query = "내일 저녁에 파스텔 공연 보고, 주말엔 호텔 체크인 날짜 바꾸고, 카페에서 체험 수업도 받고 싶어"

print("❓ 복합 질문:")
print(f"   {complex_query}\n")

print("📋 필요한 정보 (3개):")
needed = ["theater", "hotel", "cafe"]
for need in needed:
    print(f"   - {need}")

print("\n🔍 Naive RAG 검색 결과 (Top-3):\n")

# 벡터 검색
results = vectorstore.similarity_search(complex_query, k=3)

# 상세 출력
retrieved_types = []
for i, doc in enumerate(results, 1):
    doc_type = doc.metadata.get("type", "unknown")
    retrieved_types.append(doc_type)
    first_line = doc.page_content.split("\n")[0]
    print(f"{i}. [{doc_type.upper()}] {first_line}")
    print(f"   {doc.page_content[:80]}...")
    print()

# 성공/실패 체크
print("━" * 50)
print("📊 검색 결과:")
print(f"   필요: {needed}")
print(f"   검색됨: {retrieved_types}")
print()

print("💡 개별 체크:")
for need in needed:
    if need in retrieved_types:
        print(f"   ✅ {need} - 찾음")
    else:
        print(f"   ❌ {need} - 못 찾음")

found_count = sum(1 for need in needed if need in retrieved_types)
print(f"\n🎯 결과: {found_count}/3 성공 → ", end="")
if found_count == 3:
    print("✅ 완벽!")
else:
    print("❌ 실패!")

## 🎯 실습 3: 토큰 겹침 분석 - 왜 이 문서가 검색되었을까?

### 🎨 학습 목표
- 벡터 검색의 작동 원리 이해
- 키워드 겹침과 검색 결과의 관계 파악

### 💡 실습 내용
질문과 문서 사이에 **어떤 단어들이 겹치는지** 분석해봅시다!

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 다양한 질문으로 토큰 겹침을 분석해보세요

analysis_query = "파스텔 주말 이벤트"  # 원하는 질문으로 수정하세요

# 검색 수행
results = vectorstore.similarity_search(analysis_query, k=5)

print(f"🔍 질문: '{analysis_query}'\n")
print(f"📝 질문 토큰: {analysis_query.split()}\n")
print("="*70)

query_tokens = set(analysis_query.split())

for i, doc in enumerate(results, 1):
    doc_type = doc.metadata.get('type', 'unknown')
    content = doc.page_content.replace('\n', ' ')
    doc_tokens = set(content.split())
    
    # 겹치는 토큰 찾기
    overlap = query_tokens & doc_tokens
    overlap_ratio = len(overlap) / len(query_tokens) if query_tokens else 0
    
    print(f"\n{i}위. [{doc_type}]")
    print(f"  겹치는 단어: {overlap if overlap else '없음'}")
    print(f"  겹침 비율: {overlap_ratio:.1%} ({len(overlap)}/{len(query_tokens)})")
    print(f"  문서 내용: {content[:60]}...")

print("\n" + "="*70)
print("\n💡 관찰 포인트:")
print("  - 토큰이 많이 겹칠수록 상위권에 있나요?")
print("  - 토큰은 안 겹치지만 상위권인 문서가 있나요? (의미적 유사성)")
print("  - '주말', '예약' 같은 일반 키워드가 노이즈를 유발하나요?")

## 🎯 실습 4: 실패 케이스 예측 게임

### 🎨 학습 목표
- Naive RAG의 한계를 스스로 예측하기
- 실패 패턴 이해하기

### 💡 실습 내용
각 질문이 **성공할지 실패할지** 먼저 예측하고, 실제 결과와 비교해보세요!

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 각 케이스의 "예측"과 "이유"를 작성한 후 실행해보세요

test_cases = [
    {
        "query": "파스텔",
        "예측": "실패",  # "성공" 또는 "실패"
        "이유": "너무 모호해서 호텔/카페/소극장 중 무엇을 원하는지 모름",
        "원하는_결과": "명확한 1개를 선택할 수 없음"
    },
    {
        "query": "공연 보고 호텔 예약 변경하고 카페 체험하기",
        "예측": "여기에 성공/실패 예측",  # TODO
        "이유": "여기에 이유 작성",  # TODO
        "원하는_결과": "theater, hotel, cafe 3개 모두 검색"
    },
    {
        "query": "주말 이벤트",
        "예측": "여기에 성공/실패 예측",  # TODO
        "이유": "여기에 이유 작성",  # TODO
        "원하는_결과": "호텔/카페/소극장 중 하나라도 찾으면 성공"
    },
    # TODO: 여러분만의 테스트 케이스를 추가해보세요!
    {
        "query": "여기에 질문 작성",
        "예측": "성공 또는 실패",
        "이유": "예측 이유",
        "원하는_결과": "기대하는 검색 결과"
    }
]

print("🎮 실패 케이스 예측 게임\n")
print("="*70)

for i, case in enumerate(test_cases, 1):
    query = case["query"]
    prediction = case["예측"]
    reason = case["이유"]
    
    # 실제 검색
    results = vectorstore.similarity_search(query, k=3)
    retrieved_types = [doc.metadata.get('type') for doc in results]
    
    print(f"\n케이스 {i}: {query}")
    print(f"  📝 여러분의 예측: {prediction}")
    print(f"  💭 예측 이유: {reason}")
    print(f"  🔍 실제 검색 결과: {', '.join(retrieved_types)}")
    print(f"  🎯 원하는 결과: {case['원하는_결과']}")
    print(f"  ❓ 실제 성공 여부를 판단해보세요!")

print("\n" + "="*70)
print("\n💡 생각해보기:")
print("  - 예측이 맞았나요? 틀렸다면 왜 틀렸을까요?")
print("  - 어떤 질문 패턴이 실패하기 쉬운가요?")
print("  - 이 문제들을 해결하려면 어떻게 해야 할까요?")

## 🎯 실습 5: 질문 개선 실습 (Query Refinement)

### 🎨 학습 목표
- 같은 의도라도 **질문 방식**에 따라 검색 결과가 달라짐을 체험
- 효과적인 질문 작성 기법 이해

### 💡 실습 내용
**나쁜 질문 vs 좋은 질문**을 비교하고, 여러분만의 질문을 개선해보세요!

**참고**: 나중에 LLM을 사용한 자동 쿼리 개선(Query Rewriting)도 배울 예정입니다!

In [ ]:
# 질문 개선 실습
query_experiments = [
    {
        "나쁜_질문": "파스텔",
        "좋은_질문": "파스텔 호텔 예약 변경 방법",
        "개선_포인트": "구체적인 의도 명시 (호텔 + 예약 변경)"
    },
    {
        "나쁜_질문": "주말",
        "좋은_질문": "주말에 파스텔 카페에서 체험 수업 시간",
        "개선_포인트": "대상 + 행동 명확화 (카페 + 체험 수업)"
    },
    # 여기를 수정하세요! 👇
    # TODO: 여러분만의 질문 개선 사례를 추가해보세요
    {
        "나쁜_질문": "여기에 모호한 질문 작성",
        "좋은_질문": "여기에 개선된 질문 작성",
        "개선_포인트": "어떤 점이 개선되었는지 설명"
    }
]

print("🎨 질문 개선 실습 결과\n")
print("="*60)

for i, exp in enumerate(query_experiments, 1):
    print(f"\n케이스 {i}:")
    print(f"  ❌ 나쁜 질문: {exp['나쁜_질문']}")
    print(f"  ✅ 좋은 질문: {exp['좋은_질문']}")
    print(f"  💡 개선 포인트: {exp['개선_포인트']}")
    
    # 실제 검색 비교
    bad_results = vectorstore.similarity_search(exp['나쁜_질문'], k=1)
    good_results = vectorstore.similarity_search(exp['좋은_질문'], k=1)
    
    bad_type = bad_results[0].metadata.get('type', 'unknown')
    good_type = good_results[0].metadata.get('type', 'unknown')
    
    print(f"  📊 나쁜 질문 검색: [{bad_type}]")
    print(f"  📊 좋은 질문 검색: [{good_type}]")
    
    if bad_type != good_type:
        print(f"  🎯 결과: 질문 개선으로 다른 문서 검색됨! ✨")
    else:
        print(f"  ⚠️ 결과: 같은 문서 검색 (더 개선 필요)")

print("\n" + "="*60)
print("\n💡 질문 개선 팁:")
print("  1. 구체적인 키워드 사용 (예: '파스텔' → '파스텔 호텔')")
print("  2. 의도를 명확히 (예: '주말' → '주말 체험 수업 시간')")
print("  3. 컨텍스트 포함 (예: '예약' → '호텔 체크인 날짜 변경')")
print("  4. 불필요한 수식어 제거 (예: '좀', '한번', '그냥' 등)")

In [ ]:
# 전체 성능 비교 (성공 vs 실패)
test_cases = [
    {"query": "파스텔 호텔 예약 변경", "needed": ["hotel"], "category": "단순"},
    {"query": "주말 미술관 시간", "needed": ["museum"], "category": "단순"},
    {"query": complex_query, "needed": ["theater", "hotel", "cafe"], "category": "복합"}
]

results_table = []

for case in test_cases:
    query = case["query"]
    needed = case["needed"]
    category = case["category"]
    
    # 검색
    docs = vectorstore.similarity_search(query, k=3)
    found_types = [doc.metadata.get("type") for doc in docs]
    
    # 완성도 체크
    found_all = all(need in found_types for need in needed)
    
    results_table.append({
        "유형": category,
        "질문": query[:30] + "..." if len(query) > 30 else query,
        "필요": ",".join(needed),
        "검색됨": ",".join(found_types),
        "완성도": "✅" if found_all else "❌"
    })

# 표로 출력
df = pd.DataFrame(results_table)
print("📊 Naive RAG 성능 비교:\n")
print(df.to_string(index=False))

# 유형별 성공률
simple_success = sum(1 for r in results_table[:2] if r["완성도"] == "✅") / 2
complex_success = sum(1 for r in results_table[2:] if r["완성도"] == "✅") / 1

print(f"\n📈 성공률:")
print(f"   단순 질문: {simple_success:.0%} ✅")
print(f"   복합 질문: {complex_success:.0%} ❌")

## 🔍 왜 실패했을까?

위 테스트에서 **theater, hotel, cafe 3개 중 일부만** 검색되거나 **아예 다른 문서**가 검색되었습니다.

### Naive RAG의 3가지 근본적 한계

#### 1️⃣ 동음이의어 문제
- **"파스텔"** → 호텔? 카페? 소극장? 
- 벡터 검색은 **맥락 이해 부족**
- "공연", "호텔", "카페" 각각을 독립적으로만 처리

#### 2️⃣ 멀티홉 추론 실패  
- 질문 = **3개 요구사항** (공연 + 호텔 + 카페)
- Naive RAG = **단일 검색**만 수행
- Top-3로는 **3개 정보를 모두 못 찾음**

#### 3️⃣ 노이즈 문서 간섭
- 관련 없는 문서(교통, 날씨, 가이드)가 **상위권에 끼어듦**
- "주말", "예약" 같은 **일반 키워드에 반응**
- 진짜 필요한 문서가 **밀려남**

---

### 📊 성능 요약

## 🚀 다음 단계: Advanced RAG로 개선하기

### 💡 핵심 인사이트

**Naive RAG는 "검색만" 합니다.**
- 질문 분해? ❌
- 맥락 이해? ❌
- 정보 연결? ❌
- 노이즈 제거? ❌

**Advanced RAG는 "생각합니다".**
- 질문 분해 ✅
- 맥락 이해 ✅
- 정보 연결 ✅
- 노이즈 제거 ✅

### 🎯 이후 학습 내용

앞으로 다음과 같은 고급 기법들을 배우게 됩니다:

- **실패 원인 분석**: 5가지 실패 패턴 심층 분석
- **질문 분해**: Multi-Query로 복합 질문 해결
- **메타데이터 활용**: 동음이의어 문제 해결
- **검색 정확도**: Hybrid Search + Re-ranking
- **통합 시스템**: Modular RAG + Routing

**🚀 다음 노트북에서 실패 원인을 더 깊이 파헤쳐봅시다!**